In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import ast
import re
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### **Data Gathering**

In [2]:
# data collection
movies = pd.read_csv("dataset/tmdb_5000_movies.csv") 
credits = pd.read_csv("dataset/tmdb_5000_credits.csv")

In [3]:
# getting first 3 rows of movies dataset
# movies[movies['title'] == 'Harry Potter and the Half-Blood Prince']
# movies['tags'][8]
movies.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
# getting first 3 rows of credit dataset
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


* Merging two dataset movies & credits for pre-processing.

In [5]:
# merging two dataset by title
movies = movies.merge(credits, on="title")

In [6]:
# merged dataset
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


* Now we have to extract features which we will use to recommend the movie. For example, We will use `title`, `generes`, `keywords` are important feature to recommend the movie.

* Features we are going to select for tags:
    * `movie_id`
    * `title`
    * `overview`
    * `generes`
    * `keywords`
    * `cast`
    * `crew`

In [7]:
# Feature selections
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

* Now we are going to use these features and make a complete pre-processed dataset.
* Our goal is to make dataset which contain `movie_id`, `title` & `tags`

#### **Pre-Processing**

In [8]:
# checking for the null values first
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

* We just have **3 null values** in **overview** column so it is better to drop it rather taking care of. 

In [9]:
# dropping null values
movies.dropna(inplace=True)
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.iloc[0] # seeing the fist row of the dataset

movie_id                                                19995
title                                                  Avatar
overview    In the 22nd century, a paraplegic Marine is di...
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
Name: 0, dtype: object

* As we can see out data is a list of a dictionaries in which we have different data.
* This data is difficult to use so we are going to simplify the columns.
* From this:
* **genres:** `[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]`
* To this:
* **genres:** `['Action', 'Adventure', 'Fantasy', 'Science Fiction']`.
* we will extract genres names from the dictionary and make one single list.

* We can create a function which will loop through the list and extract the info.

In [11]:
# Creating a function to simplify the columns
def simplify_genres_keywords(object):
    list = []
    for i in ast.literal_eval(object):
        list.append(i['name'])
    return list

# NOTE: Our list of dictionary is in a string so first we need to convert it into an object using 'ast' library.

* Now, we'll apply this function on applicable columns which are `genres`, `keyword`.

In [12]:
movies['genres'] = movies['genres'].apply(simplify_genres_keywords)
movies['keywords'] = movies['keywords'].apply(simplify_genres_keywords)

In [13]:
# now lest check the result
movies.head(2)


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


* We also need to take care of cast and crew columns.
* We'll make one function for that as well

* `Cast` Column: Our goal is to extract first few names of the cast which will be the actors name. Because prime cast name are in the beginning of the column we will only extract those.
* From this:
* **cast column**: `[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, ...]`
* to this:
* **cast column**: `['Jack Sully', 'Sam Worthington', 'Zoe Saldana']`


* **Note**: we need to extract name of the actor not name of the character


In [14]:
# function to simplify the cast and crew columns
def simplify_cast(object):
    list = []
    counter = 0
    for i in ast.literal_eval(object):
        if counter != 3:
            list.append(i['name'])
            counter += 1 # Just to get first 3 names of the cast
        else:
            break
    return list

In [15]:
# Apply on the column
movies['cast'] = movies['cast'].apply(simplify_cast)

In [16]:
# Lest check the result
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


* now we will do similar with crew column. We wil focus on getting **director name**. from the column we will extract name of the crew where job name is director.
* We won't be focusing on person with job name is not director, such as **'cameraman', 'designer', 'casting'** etc.

In [17]:
# function to fetch name of the director from crew column
def simplify_crew(object):
    list = []
    for i in ast.literal_eval(object):
        if i['job'] == 'Director':
            list.append(i['name'])
    return list

In [18]:
# now lets apply it to our column.
movies['crew'] = movies['crew'].apply(simplify_crew)

In [19]:
#result
movies.head(2)


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


* Now, to avoid the redundancy and maintain data conflict of the words we will remove space between words such as `Sam Worthington` to `SamWorthington`. We will use simple labmda function on `genres`, `keywords`, `cast` & `crew` columns

In [20]:
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

In [21]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [22]:
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]


* `preprocess_text()` will be used for processing text in usable format.

In [23]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove spaces and special characters
    text = re.sub(r'[^a-zA-Z0-9]', '', text)
    return text

* Our goal is to make tags and we will use all these columns to make tags. So from this data we will create new column called `tags` in which we will combine all the columns except `movie_id` and `title`.
* Creating `tag` column and concatinating the columns. 

In [24]:
# Concatenate columns
movies['tags'] = movies['overview'] + movies['genres']+ movies['keywords']+ movies['cast']+ movies['crew']
movies['cast'] = movies['cast'] + movies['crew']
movies['tags']= movies['tags'].apply(lambda x:" ".join(x))
movies['tags'] = movies['tags'] + " "  + movies['title'].apply(preprocess_text)
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [25]:
movies.head(5)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor...",[GoreVerbinski],"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, SamM...",[SamMendes],a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Chri...",[ChristopherNolan],following the death of district attorney harve...
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, An...",[AndrewStanton],"john carter is a war-weary, former military ca..."


* As we have got the `tags` we needed we can drop other columns or we can make new data frame.

In [26]:
movies.drop(['overview', 'genres', 'keywords', 'cast', 'crew'], axis=1, inplace=True)
movies.head(3)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...


### Text Preparation for NLP

* We need to convert text in usable  format, We will convert words into base form/remove prefix etc. using the `PorterStemmer` from `nltk` library 

In [27]:
# Making instance of PorterStemmer()
porter_steam = PorterStemmer()

In [28]:
# Making a function which will apply steamming on text
def apply_steamming(text):
    temp_list = []
    #split string and apply steamming in words then join again
    for word in text.split():
        temp_list.append(porter_steam.stem(word))
    return " ".join(temp_list)

In [29]:
# Applying steamming on `tags` column.
movies['tags'] = movies['tags'].apply(apply_steamming)

In [30]:
movies.head(5)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [31]:
movies[movies['title'] == 'The Avengers']

,movie_id,title,tags
16,24428,The Avengers,when an unexpect enemi emerg and threaten glob...


### TF-IDF

* Next step is to make vectors for each movies Using TF-IDF.

* Now as we have data frame prepared we will apply NLP on it to make `vectors` of each movie and we will calculate cosine similarity to generate some recommendations.
**Processing User's input:**
    * To search for specific input from user  we will process the input and will search it in tags column. If we get the desired result we will generate recommendatoins based on that.

In [32]:
#Creating vectors for alll the movies.
vectorizer = TfidfVectorizer(stop_words='english')
vector = vectorizer.fit_transform(movies['tags'])
cosine_similarity = cosine_similarity(vector)

In [90]:
# function to generate recommendations which will return list of recommendations.
def generate_recommendations(user_input):
    
    #list of recommendations
    recommendation_list = {}
    #prepare input
    processed_movie_name = preprocess_text(user_input)
    
    # Search for the user input in the 'tags' column
    for index, row in movies.iterrows():
        if processed_movie_name in row['tags']:

            # Get the title of the movie found in the tags column
            movie_title = row['title']
            
            # Compute cosine similarity for the movie
            index = movies[movies['title'] == movie_title].index[0]
            similar_movies = sorted(enumerate(cosine_similarity[index]), reverse=True, key=lambda x: x[1])[:20]
            
            # append names in list and return the list
            for i in similar_movies:
                movie_id = movies.iloc[i[0]].movie_id
                movie_name = movies.iloc[i[0]].title
                if movie_id not in recommendation_list:
                    recommendation_list[movie_id] = movie_name
                else:
                    continue
            return recommendation_list

    # If the movie is not found in the 'tags' column
    print(f"No similar movies found for '{user_input}' in the dataset.")

In [91]:
generate_recommendations("Robert Downey Jr.")

{99861: 'Avengers: Age of Ultron',
 24428: 'The Avengers',
 10138: 'Iron Man 2',
 271110: 'Captain America: Civil War',
 1726: 'Iron Man',
 68721: 'Iron Man 3',
 10195: 'Thor',
 2080: 'X-Men Origins: Wolverine',
 76170: 'The Wolverine',
 76338: 'Thor: The Dark World',
 100402: 'Captain America: The Winter Soldier',
 102899: 'Ant-Man',
 246655: 'X-Men: Apocalypse',
 166424: 'Fantastic Four',
 36657: 'X-Men',
 1771: 'Captain America: The First Avenger',
 127585: 'X-Men: Days of Future Past',
 102382: 'The Amazing Spider-Man 2',
 1724: 'The Incredible Hulk',
 118340: 'Guardians of the Galaxy'}